# Create animations using matplotlib

Artist Animation is nice because you make consecutive figures and simply append them to a list. To save as a .mp4 ffmpeg is required. ffmpeg can be downloaded here: https://ffmpeg.org/download.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from IPython.display import HTML
from pathlib import Path

# direct matplotlib to ffmpeg executable, or add to path and skip
plt.rcParams['animation.ffmpeg_path'] = r'/Users/michaelfield/ffmpeg'
#plt.rcParams['animation.ffmpeg_path'] = r'C:\\ffmpeg\\bin\\ffmpeg.exe'

# use jshtml rendering, otherwise call HTML(ani.to_jshtml()) for ArtistAnimation and FuncAnimation
plt.rcParams['animation.html'] = 'jshtml'

## 2D gaussian setup

In [2]:
# x and y 2D space
x = np.linspace(0, 1, 50)
y = np.linspace(0, 1, 50)

xx, yy = np.meshgrid(x, y)

# sigma of gaussian
s = 0.1

# x and y mean of gaussian to iterate over
xbs = np.linspace(0, 1, 20)
ybs = np.linspace(0.2, 0.8, 20)

## .mp4 using ArtistAnimation and ffmpeg backend

In [3]:
# create single subplot
fig, ax = plt.subplots(figsize=(8,8))

# create colorbar axis
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes('right', size='5%', pad='2%')

ax.tick_params(labelsize=14)

ims = []

for xb, yb in zip(xbs, ybs):
    # compute 2D gaussian
    zz = 1/2/np.pi/s**2*np.exp(-1*((xx-xb)**2+(yy-yb)**2)/2/s**2)
    
    # pcolormesh or imshow
    im = ax.pcolormesh(xx, yy, zz, vmin=0, vmax=15, shading='auto')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_title('2D moving Gaussian', fontsize=20)

    # colorbar
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label('intensity', size=16)
    
    # must append as list
    ims.append([im])

# create animation and save, adjust interval
ani = animation.ArtistAnimation(fig, ims, interval=25)
ani.save(Path('output/gaussian.mp4'), dpi=300)
plt.close()

# display using html
ani

## .gif using PillowWriter

In [4]:
# create single subplot
fig, ax = plt.subplots(figsize=(8,8))

# create colorbar axis
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes('right', size='5%', pad='2%')

ax.tick_params(labelsize=14)

ani = animation.PillowWriter(fps=50)
ani.setup(fig, Path('output/pillow.gif'), dpi=300)

for xb, yb in zip(xbs, ybs):
    # compute 2D gaussian
    zz = 1/2/np.pi/s**2*np.exp(-1*((xx-xb)**2+(yy-yb)**2)/2/s**2)
    
    # pcolormesh or imshow
    im = ax.pcolormesh(xx, yy, zz, vmin=0, vmax=15, shading='auto')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_title('2D moving Gaussian', fontsize=20)

    # colorbar
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label('intensity', size=16)
    
    ani.grab_frame()

# create animation and save, adjust interval

ani.finish()
plt.close()

<img src="pillow.gif" width="600" align="center">

## .html using HTMLWriter

In [8]:
# create single subplot
fig, ax = plt.subplots(figsize=(8,8))

# create colorbar axis
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes('right', size='5%', pad='2%')

ax.tick_params(labelsize=14)

ani = animation.HTMLWriter(fps=50)
ani.setup(fig, Path('output/gaussian.html'), dpi=300)

for xb, yb in zip(xbs, ybs):
    # compute 2D gaussian
    zz = 1/2/np.pi/s**2*np.exp(-1*((xx-xb)**2+(yy-yb)**2)/2/s**2)
    
    # pcolormesh or imshow
    im = ax.pcolormesh(xx, yy, zz, vmin=0, vmax=15, shading='auto')
    ax.set_xlabel('X', fontsize=16)
    ax.set_ylabel('Y', fontsize=16)
    ax.set_title('2D moving Gaussian', fontsize=20)

    # colorbar
    cbar = plt.colorbar(im, cax=cax)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label('intensity', size=16)
    
    ani.grab_frame()

# create animation and save, adjust interval

ani.finish()
plt.close()

HTML(str(Path('output/gaussian.html')))

# My preferred method
# .mp4 using FuncAnimation with ffmpeg backend

Move all colorbar stuff out of animate function due to bug that creates extra plot

In [9]:
%%time

zstack = np.zeros((x.size, y.size, xbs.size))
for i, (xb, yb) in enumerate(zip(xbs, ybs)):
    zz = 1/2/np.pi/s**2*np.exp(-1*((xx-xb)**2+(yy-yb)**2)/2/s**2)
    zstack[:,:,i] = zz

# create single subplot
fig, ax = plt.subplots(figsize=(8,8))

# create colorbar axis
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes('right', size='5%', pad='2%')

ax.tick_params(labelsize=14)

###... initialize plot with colorbar
im = ax.pcolormesh(xx, yy, zstack[:,:,0], vmin=0, vmax=15, shading='auto')
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.set_title('2D moving Gaussian', fontsize=20)

# colorbar
cbar = plt.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=14)
cbar.set_label('intensity', size=16)

def animate(i):
    # udpate the array instead of rendering a new pcolormesh for speed
    im.set_array(zstack[:,:,i])
    
ani = animation.FuncAnimation(fig, animate, frames=zstack.shape[2], interval=25)
ani.save(Path('output/gaussian.mp4'), dpi=300)
plt.close()

# display
ani

CPU times: user 809 ms, sys: 126 ms, total: 935 ms
Wall time: 1.76 s
